In [1]:
#Import Dependencies
%matplotlib inline
import os
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import json
from pprint import pprint
from datetime import datetime
from datetime import timedelta 
import time
import matplotlib.pyplot as plt

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
#Variables
start_date = "2008-01-01"
initial_date = datetime(2008, 1, 1)
end_date = (datetime.today() - timedelta(days = 2)).date()
print(str(initial_date.date()))

2008-01-01


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

In [5]:
#Banxico URL
url = 'https://www.banxico.org.mx/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria/anuncios-politica-monetaria-t.html'
browser.visit(url)
api_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"
banxico_key = "85c8edeab72128102cdad764f0a2d4532c336d22690a51104b500fb6f077ed4c"

In [6]:
html = browser.html
soup = bs(html, 'html.parser')

In [7]:
links = []
for link in soup.find_all('a'):
    if ('/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria') in link.get('href'):
        partial_link = link.get('href')
        links.append('http://www.banxico.org.mx' + partial_link)
links = links[1:]
#links

In [8]:
dates = []
for date in soup.find_all("td"):
    data = date.get_text()
    dates.append(data)
del dates[1::2]

In [9]:
import re
dates2 = []
for index in range(len(dates)):
    dates2.append(re.sub("\s", "", dates[index])) 


In [10]:
dates3 = []
for index in range(len(dates)):
    dates3.append(re.sub("[\/]", "", dates2[index])) 
#print(dates3)

In [11]:
name_counter = 0
for i in links:
    r = requests.get(i)
    out_file = open(f'{dates3[name_counter]}.pdf', 'wb')
    out_file.write(r.content)
    name_counter += 1

In [12]:
#Guillermo 
#pdf to text

In [13]:
final_dates = []
for date in dates2:
    dates = datetime.strptime(date, "%d/%m/%y")
    if dates > initial_date:
        final_dates.append(datetime.strftime(dates, "%Y-%m-%d"))
final_dates.append(datetime.strftime(end_date, "%Y-%m-%d"))

In [20]:
#MXN Lvl
Series = "SF43788"

fxt = []
for fecha in final_dates:
    url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
    for query in url_consulta: 
        data = requests.get(query).json()
        fxt.append(data["bmx"]["series"][0]["datos"][0]["dato"])
time.sleep(120) #tiempo del API

In [24]:
Series = "SF43878"

tasa = []
for fecha in final_dates:
    try:
        url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
        for query in url_consulta: 
            data = requests.get(query).json()
            tasa.append(data["bmx"]["series"][0]["datos"][0]["dato"])
    except:
        tasa.append(0)
time.sleep(120) #tiempo del API   

In [25]:
Series = "SP30578"

#Total Data
total_query_url = f"{api_url}{Series}/datos/{start_date}/{end_date}?token={banxico_key}"
total_data = requests.get(total_query_url).json()
#print(total_data)
data_total = total_data['bmx']['series'][0]['datos']
data_total

[{'fecha': '01/01/2008', 'dato': '3.70000000'},
 {'fecha': '01/02/2008', 'dato': '3.72000000'},
 {'fecha': '01/03/2008', 'dato': '4.25000000'},
 {'fecha': '01/04/2008', 'dato': '4.55000000'},
 {'fecha': '01/05/2008', 'dato': '4.95000000'},
 {'fecha': '01/06/2008', 'dato': '5.26000000'},
 {'fecha': '01/07/2008', 'dato': '5.39000000'},
 {'fecha': '01/08/2008', 'dato': '5.57000000'},
 {'fecha': '01/09/2008', 'dato': '5.47000000'},
 {'fecha': '01/10/2008', 'dato': '5.78000000'},
 {'fecha': '01/11/2008', 'dato': '6.23000000'},
 {'fecha': '01/12/2008', 'dato': '6.53000000'},
 {'fecha': '01/01/2009', 'dato': '6.28000000'},
 {'fecha': '01/02/2009', 'dato': '6.20000000'},
 {'fecha': '01/03/2009', 'dato': '6.04000000'},
 {'fecha': '01/04/2009', 'dato': '6.17000000'},
 {'fecha': '01/05/2009', 'dato': '5.98000000'},
 {'fecha': '01/06/2009', 'dato': '5.74000000'},
 {'fecha': '01/07/2009', 'dato': '5.44000000'},
 {'fecha': '01/08/2009', 'dato': '5.08000000'},
 {'fecha': '01/09/2009', 'dato': '4.8900

In [26]:
#Match fechas
data_total_df = pd.DataFrame(data_total)
data_total_df['fecha'] = pd.to_datetime(data_total_df['fecha'], format='%d/%m/%Y')
data_total_df['key'] = pd.to_datetime(data_total_df['fecha']).dt.to_period('M')
data_total_df

,fecha,dato,key
0,2008-01-01,3.70000000,2008-01
1,2008-02-01,3.72000000,2008-02
2,2008-03-01,4.25000000,2008-03
3,2008-04-01,4.55000000,2008-04
4,2008-05-01,4.95000000,2008-05
...,...,...,...
143,2019-12-01,2.83000000,2019-12
144,2020-01-01,3.24000000,2020-01
145,2020-02-01,3.70000000,2020-02
146,2020-03-01,3.25000000,2020-03


In [27]:
pre_banxico_dict = {"MXN": fxt,  "Tasa": tasa}
pre_banxico_df = pd.DataFrame(pre_banxico_dict)
pre_banxico_df['Fecha'] = final_dates
pre_banxico_df['key'] = pd.to_datetime(pre_banxico_df['Fecha']).dt.to_period('M')
pre_banxico_df

,MXN,Tasa,Fecha,key
0,24.0500,0,2020-05-14,2020-05
1,24.3030,0,2020-04-21,2020-04
2,24.2160,0,2020-03-20,2020-03
3,18.6190,0,2020-02-13,2020-02
4,18.9480,0,2019-12-19,2019-12
...,...,...,...,...
106,10.4650,0,2008-04-18,2008-04
107,10.7520,0,2008-03-14,2008-03
108,10.7560,0,2008-02-15,2008-02
109,10.9160,0,2008-01-18,2008-01


In [28]:
banxico_merge = data_total_df.merge(pre_banxico_df, on='key')
banxico_merge

,fecha,dato,key,MXN,Tasa,Fecha
0,2008-01-01,3.70000000,2008-01,10.9160,0,2008-01-18
1,2008-02-01,3.72000000,2008-02,10.7560,0,2008-02-15
2,2008-03-01,4.25000000,2008-03,10.7520,0,2008-03-14
3,2008-04-01,4.55000000,2008-04,10.4650,0,2008-04-18
4,2008-05-01,4.95000000,2008-05,10.4135,0,2008-05-16
...,...,...,...,...,...,...
104,2019-11-01,2.97000000,2019-11,19.3760,0,2019-11-14
105,2019-12-01,2.83000000,2019-12,18.9480,0,2019-12-19
106,2020-02-01,3.70000000,2020-02,18.6190,0,2020-02-13
107,2020-03-01,3.25000000,2020-03,24.2160,0,2020-03-20


In [ ]:

Banxico_dict = {"MXN": fxt,  "Tasa": tasa }
                
Banxico_data = pd.DataFrame(Banxico_dict)
Banxico_data["Fecha"] = final_dates

Banxico_data = Banxico_data[['Fecha', 'MXN', "Tasa"]]
Banxico_data = Banxico_data.sort_values('Fecha')
Banxico_data = Banxico_data[Banxico_data.Tasa != 0]
Banxico_data = Banxico_data.reset_index(drop = True)

In [ ]:
#Xime Word count feed a este DF



In [ ]:

Banxico_data['Inflacion'] = 0
#Banxico_data['Crecimiento'] = 0
#Banxico_data['Desempleo'] = 0
Banxico_data['Dif'] = 0
Banxico_data['Tasa'] = Banxico_data['Tasa'].astype(float)
Banxico_data

In [ ]:
for i in range(1, len(Banxico_data) -1 ):
    Banxico_data.loc[i, 'Dif'] = 25 * round((100 * (Banxico_data.loc[i, 'Tasa'] - Banxico_data.loc[i+1, 'Tasa']))/25)

In [ ]:
Banxico_data

In [ ]:
# Assign X (data) and y (target)
X = Banxico_data.drop(["Dif", "Fecha"], axis=1)
y = Banxico_data["Dif"]
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

In [ ]:
 classifier.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:50]}")
print(f"First 10 Actual labels: {y_test[:50].tolist()}")

In [ ]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)